In [ ]:
# Plot ad hoc mnist instances
import tensorflow as tf
from keras.datasets import mnist
import matplotlib.pyplot as plt
# load (downloaded if needed) the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# plot 6 images
plt.subplot(321)
plt.imshow(x_train[0], cmap=plt.cm.binary)
plt.subplot(322)
plt.imshow(x_train[1], cmap=plt.cm.binary)
plt.subplot(323)
plt.imshow(x_train[2], cmap=plt.cm.binary)
plt.subplot(324)
plt.imshow(x_train[3], cmap=plt.cm.binary)
plt.subplot(325)
plt.imshow(x_train[4], cmap=plt.cm.binary)
plt.subplot(326)
plt.imshow(x_train[5], cmap=plt.cm.binary)
# show the plot
plt.show()

In [ ]:
print("shape of x_train data: ", x_train.shape)
print("shape of y_train data: ", y_train.shape)
print("shape of x_test data: ", x_test.shape)
print("shape of y_test data: ", y_test.shape)

#First we are going to create simple MLP 

## 1. Data Prepration

In [ ]:
# Some important libraries and APIs 
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils

# Because we have data in 3 dimesion (n,28,28) so we have convert it into (n,784) for MLP
# This is called flattening 
flat_pixels = x_train.shape[1] * x_train.shape[2] # 28*28
x_train = x_train.reshape((x_train.shape[0], flat_pixels)).astype('float32')
x_test = x_test.reshape((x_test.shape[0], flat_pixels)).astype('float32')

In [ ]:
flat_pixels

In [ ]:
print("shape of x_train data: ", x_train.shape)
print("shape of x_test data: ", x_test.shape)

In [ ]:
# In our dataset we have pixels in the range of 0-255 so we have convert it into 0-1 for simple calcuation
# This is called normalization
x_train = x_train / 255
x_test = x_test / 255

In [ ]:
y_train[0]

In [ ]:
# As we know we have 0-9 digits to classifiy 
# Means we have low categories so we can use one hot encoding here for output
# np_utils is a numpy utility where we have function called to_categorical with this we do onehot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [ ]:
classes = y_test.shape[1]
classes

In [ ]:
print("Now shape is like this 2D shape: ",y_train.shape)

## 2. Defining a model

In [ ]:
# MLP model with 1 hidden layer function 
# We are using fucntion so that we can change it easily later 
def MLP_h1_model():
	# create model
	model = Sequential()
    # Dense laye configuration
    # First arguments is shape of the output we will get from this layer
    # input_dim = size of the input we are providing
    # Kernel_initializer = Weight initializer for initial layer
    # Activation is relu here to find output for next layer
    # flat_pixels=784
	model.add(Dense(flat_pixels, input_dim=flat_pixels, kernel_initializer='normal', activation='relu'))
	model.add(Dense(classes, kernel_initializer='normal', activation='softmax'))
	# Compile model
    # https://machinelearningmastery.com/cross-entropy-for-machine-learning/ for more idea
    # Optimizer we are using adam and accuracy as a matrix
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
model = MLP_h1_model()
model.summary()

In [ ]:
# first two are input and output train ndarrays
# Validation_data = how do you want to validate your results on each epochs
# Batch size is to tell that how many inputs it needs to take on one updation of weights 
# Verbose is to how you the progress bas 0,1,2. 
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("MLP_h1_model Accuracy: %.2f%%" % (scores[1]*100))
print("MLP_h1_model Error: %.2f%%" % (100-scores[1]*100))

**Observations**
As we can see we get an 98.36% accuracy then how we can improve it 

# Let's work with CNN 

In [ ]:
# Important libraries and Apis for CNN
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
# load (downloaded if needed) the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## 1. Data Prepration
As we know Convolutional Neural network is working on the Matrices
* first, we have to conver our data into acceptable form 
* What the shape of acceptance of the CNN layer : [pixels][width][height][channels]
* so we have convert it into this shape now we have [pixels][width][height]



In [ ]:
# to laod data (x_train, y_train), (x_test, y_test) = mnist.load_data()
# reshape to be [samples][width][height][channels]
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

In [ ]:
print("New shape is: ",x_train.shape)

In [ ]:
# normalize inputs from 0-255 to 0-1
x_train = x_train / 255
x_test = x_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
classes = y_test.shape[1]

In [ ]:
x_train = x_train.reshape(len(x_train),28,28,1)
x_test = x_test.reshape(len(x_test),28,28,1)

In [ ]:
y_train.shape

In [ ]:
# simple model with one conv2D layer only 
def CNN_1L_model():
	# create model
	model = Sequential()
    # https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/
    # https://keras.io/api/layers/convolution_layers/convolution2d/ to know more
    # 32 is number of filers/kernels we are using in this layer 
    # (5,5) is the kernel size we are using here 
    # input is to tell that get each image one by one 
	model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
    # 2*2 maxpooling 
	model.add(MaxPooling2D(pool_size=(2, 2)))
    # To reduce 20% data 
	model.add(Dropout(0.2))
    # flatten will make 28*28 to 784
	model.add(Flatten())
    # These are the dense layer same as MLP
	model.add(Dense(128, activation='relu'))
    # Last layer with softmax activation
	model.add(Dense(classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
model.summary()

In [ ]:
def CNN_2L_model():
	# create model
	model = Sequential()
	model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Conv2D(15, (3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
def CNN_3L1_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu'))
	model.add(Conv2D(64, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = keras.optimizers.Adam(lr=0.01, momentum=0.9)
	model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
	return model


In [ ]:
def CNN_3L2_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu'))
	model.add(Conv2D(32, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = keras.optimizers.Adam(lr=0.01, momentum=0.9)
	model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [ ]:
def CNN_4L_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    # model.add(Conv2D(32, (3, 3), activation='relu'))
    # model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = keras.optimizers.Adam(lr=0.01, momentum=0.9)
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model1 = CNN_1L_model()
model2 = CNN_2L_model()
model31 = CNN_3L1_model()
model32 = CNN_3L2_model()
model4 = CNN_4L_model()

In [ ]:
models = [model1,model2,model31,model32,model4]

In [ ]:
x_test.shape

In [ ]:
count=0
for model in models:
    count+=1
    # Fit the model
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir = "/content/logs/layer"+str(count),histogram_freq = 1)
                                                 
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200,verbose=0,callbacks=[tb_callback])
    # Final evaluation of the m
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("\nAccuracy: %.2f%%" % (scores[1]*100))
    print("Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
count=0
for model in models:
    count+=1
    # Fit the model
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir = "/content/logs1/layer"+str(count),histogram_freq = 1)
                                                 
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200,verbose=0,callbacks=[tb_callback])
    # Final evaluation of the m
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("\nAccuracy: %.2f%%" % (scores[1]*100))
    print("Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
count=0
for model in models:
    count+=1
    # Fit the model
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir = "/content/logs3/layer"+str(count),histogram_freq = 1)
                                                 
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=200,verbose=0,callbacks=[tb_callback])
    # Final evaluation of the m
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("\nAccuracy: %.2f%%" % (scores[1]*100))
    print("Error: %.2f%%" % (100-scores[1]*100))

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/logs1/

**Observations**<br>
After 5 time running 5 models we get our second model is doing we 

In [ ]:
model2.summary()